In [1]:
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.sql import functions as F

In [2]:
#create enviroment
spark = SparkSession.builder.getOrCreate()

23/05/22 12:30:18 WARN Utils: Your hostname, Carolines-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.4 instead (on interface en0)
23/05/22 12:30:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/22 12:30:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### 1. Read the case, department, and source data into their own spark dataframes.

In [3]:
df = pd.read_csv('311 - 311.csv')

In [4]:
df = spark.createDataFrame(df)

#### 2. Let's see how writing to the local disk works in spark:

Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [5]:
#df.write.type
df.write.json('data/sources_json', mode='overwrite')

23/05/22 12:30:29 WARN TaskSetManager: Stage 0 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [6]:
#df.write.format()
(
df.write.format('csv')
    .mode('overwrite')
    .option('header', 'True')
    .save('data/sources_csv')
)

23/05/22 12:30:33 WARN TaskSetManager: Stage 1 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [7]:
df.show(3, vertical=True, truncate=False)

23/05/22 12:30:35 WARN TaskSetManager: Stage 2 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 1/1/18 0:42                           
 case_closed_date     | 1/1/18 12:29                          
 SLA_due_date         | 9/26/20 0:42                          
 case_late            | NO                                    
 num_days_late        | -998.5087616                          
 case_closed          | YES                                   
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
-RECORD 1----------------------------------------------

In [8]:
df.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

**Correcting Data Types**

case_opened_date --> datetime

case_closed_date --> datetime

SLA_due_date --> datetime

case_late --> binary (1, 0)

case_closed --> binary (1, 0)

dept_division --> lower                        

service_request_type --> lower

case-status --> binary (1, 0)

request_address --> lower and split address into street and number and city and zip

In [9]:
df = df.withColumnRenamed('SLA_due_date', 'case_due_date')

In [10]:
fmt = 'M/d/yy H:m'

In [11]:
#use to_timestamp
df = df.withColumn('case_opened_date',
    F.to_timestamp('case_opened_date', fmt))

In [12]:
#use to_timestamp
df = df.withColumn('case_closed_date',
    F.to_timestamp('case_closed_date', fmt))

In [13]:
#use to_timestamp
df = df.withColumn('case_due_date',
    F.to_timestamp('case_due_date', fmt))

In [14]:
df.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [15]:
#use condition to make true and false
df = df.withColumn(
    'case_closed',
    df.case_closed == 'YES'
    # F.expr('case_closed == "YES"')
).withColumn(
    'case_late',
    F.expr('case_late == "YES"')
)

In [16]:
df.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [17]:
df.select('case_status').distinct().show()

23/05/22 12:30:36 WARN TaskSetManager: Stage 3 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-----------+
|case_status|
+-----------+
|       Open|
|     Closed|
+-----------+



In [18]:
df = df.withColumn(
    'case_status',
    df.case_status == 'Open'
)

In [19]:
df.select('case_status').distinct().show()

23/05/22 12:30:38 WARN TaskSetManager: Stage 6 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-----------+
|case_status|
+-----------+
|       true|
|      false|
+-----------+



In [20]:
df.dtypes


[('case_id', 'bigint'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'boolean'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [21]:
df = df.withColumn(
    'request_address',
    F.trim(F.lower('request_address'))
    ).withColumn(
    'service_request_type',
    F.trim(F.lower('service_request_type'))
    ).withColumn(
    'dept_division',
    F.trim(F.lower('dept_division'))
    )

In [22]:
df.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'boolean'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [23]:
df.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616                         
 case_closed          | true                                 
 dept_division        | field operations                     
 service_request_type | stray animal                         
 SLA_days             | 999.0                                
 case_status          | false                                
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 5                                    
-RECORD 1----------------------------------------------------
 case_id

23/05/22 12:30:39 WARN TaskSetManager: Stage 9 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [24]:
df = df.withColumn(
    'zip_code',
    F.regexp_extract('request_address',
                     '(\d+$)',1)
)

In [25]:
# df = df.withColumn(
#     'request_address',
#     F.regexp_extract('request_address',
#                      '((\d+?)\s+?(\w+)\s+?(\w+))', 1)
# )

df = df.withColumn(
    'request_address',
    F.regexp_extract('request_address',
                     '^(.+?),', 1)
)

In [26]:
df.select('request_address').show(2)

+----------------+
| request_address|
+----------------+
|2315  el paso st|
| 2215  goliad rd|
+----------------+
only showing top 2 rows



23/05/22 12:30:39 WARN TaskSetManager: Stage 10 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [27]:
# Finished with the clean!

#### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [28]:
df.filter(
    df.case_closed == 'false'
).filter(
    df.SLA_days != 'NaN'
).sort(desc('SLA_days')).show(1, vertical=True, truncate=False)

# ~1405 days late

23/05/22 12:30:40 WARN TaskSetManager: Stage 11 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------------------------
 case_id              | 1014178781                             
 case_opened_date     | 2018-01-19 17:07:00                    
 case_closed_date     | null                                   
 case_due_date        | 2021-11-25 08:30:00                    
 case_late            | false                                  
 num_days_late        | -1204.354167                           
 case_closed          | false                                  
 dept_division        | signals                                
 service_request_type | signal timing modification by engineer 
 SLA_days             | 1405.640856                            
 case_status          | true                                   
 source_id            | CRM_Listener                           
 request_address      |                                        
 council_district     | 10                                     
 zip_code             |                 

In [29]:
#use datediff() to find the difference between two dates
df = df.withColumn(
    'case_age',
    F.datediff(
        F.current_timestamp(),
        'case_opened_date'
    )
)

In [30]:
df.filter(
    df.case_closed == 'false'
).sort(desc('case_age')).show(1, vertical=True, truncate=False)

# ~1966 days late

23/05/22 12:30:41 WARN TaskSetManager: Stage 12 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0-----------------------------------
 case_id              | 1014128388          
 case_opened_date     | 2018-01-02 09:39:00 
 case_closed_date     | null                
 case_due_date        | 2018-01-09 09:39:00 
 case_late            | true                
 num_days_late        | 211.5974884         
 case_closed          | false               
 dept_division        | 311 call center     
 service_request_type | complaint           
 SLA_days             | 7.0                 
 case_status          | true                
 source_id            | mt13131             
 request_address      | 7326  westglade pl  
 council_district     | 6                   
 zip_code             | 78227               
 case_age             | 1966                
only showing top 1 row



#### 2. How many Stray Animal cases are there?

In [31]:
df.filter(df.service_request_type == 'stray animal').count()

23/05/22 12:30:42 WARN TaskSetManager: Stage 13 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


3497

#### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [32]:
df.filter(
    (df.dept_division == 'field_operations') & 
(df.service_request_type != 'officer standby')
).count()

23/05/22 12:30:43 WARN TaskSetManager: Stage 16 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


0

#### 4. Convert the council_district column to a string column.

In [33]:
#use .cast()
df = df.withColumn('council_district',
             F.col('council_district').cast('string'))

#### 5. Extract the year from the case_closed_date column.

In [34]:
df.withColumn(
    'year',
    F.year(F.col('case_closed_date'))
).show(3, vertical=True, truncate=False)

-RECORD 0--------------------------------------
 case_id              | 1014127332             
 case_opened_date     | 2018-01-01 00:42:00    
 case_closed_date     | 2018-01-01 12:29:00    
 case_due_date        | 2020-09-26 00:42:00    
 case_late            | false                  
 num_days_late        | -998.5087616           
 case_closed          | true                   
 dept_division        | field operations       
 service_request_type | stray animal           
 SLA_days             | 999.0                  
 case_status          | false                  
 source_id            | svcCRMLS               
 request_address      | 2315  el paso st       
 council_district     | 5                      
 zip_code             | 78207                  
 case_age             | 1967                   
 year                 | 2018                   
-RECORD 1--------------------------------------
 case_id              | 1014127333             
 case_opened_date     | 2018-01-01 00:46

23/05/22 12:30:44 WARN TaskSetManager: Stage 19 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


#### 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [35]:
df.withColumn(
    'num_hours_late',
    (df.num_days_late * 24)
).show(3, vertical=True, truncate=False)

-RECORD 0--------------------------------------
 case_id              | 1014127332             
 case_opened_date     | 2018-01-01 00:42:00    
 case_closed_date     | 2018-01-01 12:29:00    
 case_due_date        | 2020-09-26 00:42:00    
 case_late            | false                  
 num_days_late        | -998.5087616           
 case_closed          | true                   
 dept_division        | field operations       
 service_request_type | stray animal           
 SLA_days             | 999.0                  
 case_status          | false                  
 source_id            | svcCRMLS               
 request_address      | 2315  el paso st       
 council_district     | 5                      
 zip_code             | 78207                  
 case_age             | 1967                   
 num_hours_late       | -23964.210278399998    
-RECORD 1--------------------------------------
 case_id              | 1014127333             
 case_opened_date     | 2018-01-01 00:46

23/05/22 12:30:45 WARN TaskSetManager: Stage 20 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


#### 7. Join the case data with the source and department data.

In [36]:
dept_df = pd.read_csv('dept - dept.csv')

In [37]:
dept_df = spark.createDataFrame(dept_df)

In [38]:
dept_df = dept_df.withColumn(
    'dept_division',
    F.trim(F.lower('dept_division'))
    ).withColumn(
    'dept_name',
    F.trim(F.lower('dept_name'))
    ).withColumn(
    'standardized_dept_name',
    F.trim(F.lower('standardized_dept_name'))
    ).withColumn(
    'dept_subject_to_SLA',
    F.trim(F.lower('dept_subject_to_SLA'))
)

In [39]:
df = df.join(dept_df, how='left', on ='dept_division')

#### 8. Are there any cases that do not have a request source?

In [45]:
df.groupBy(df.source_id).count().show(100)

23/05/22 13:31:01 WARN TaskSetManager: Stage 33 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+------------+-----+
|   source_id|count|
+------------+-----+
|      136202|  417|
|     MW16328| 1329|
|      141239|  872|
|     df03076|    5|
|      141549|  476|
|     mp21218| 2005|
|     sp26368|   23|
|     bo26471|    5|
|     sg22264|  281|
|     js12254|    5|
|     bn26322|   33|
|     BA10591|    4|
|     ns16326| 2090|
|     MB16118|   31|
|     rs16746|  359|
|      141256|   70|
|     ag19640|    1|
|     cc17850|  265|
|     RG26608|   16|
|     me05816| 2147|
|     sw26367|   28|
|      139342| 2328|
|      141954|    1|
|      120752| 2060|
|     dl05036| 1369|
|     NO10960| 2348|
|     FR26584|   15|
|     WT26601|    7|
|     mc21309| 2012|
|     DG26131|   20|
|     JB26299|   19|
|     mt26294|   14|
|     ss09159| 1689|
|     BM26349|    3|
|     af26445|   32|
|      141752|    4|
|     dl09223|    3|
|     dv26072|    3|
|      139344| 2385|
|      138793| 2272|
|     FC24472|   13|
|      142738|  541|
|      139868| 1824|
|     ss21394| 1962|
|      140987

#### 9. What are the top 10 service request types in terms of number of requests?

AttributeError: 'DataFrame' object has no attribute 'desc'